<a href="https://colab.research.google.com/github/Zypperman/DBTT_G1_GRP3/blob/prelim_EDA/Usecase_2_1_(EcStateFactored).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Case 2.1: CWP with Time-series economic data
(For capacity building)

# As mentioned in the readme for the github repo, please note that the content for this notebook was largely adapted from the [AMEX Credit Score Model established by Kaggle user Gopi Durgaprasad](https://www.kaggle.com/code/gopidurgaprasad/amex-credit-score-model). As it largely replicates the expected utility for Use case 2.0, we will not implement it but extend its functionality here, as we add to its calculations.

## Problem statement

- A Creditworthiness score is how officers are able to tell, with respect to an applicant's history, whether

# **<span style="color:#F7B2B0;">Data</span>**

The objective of this competition is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile. The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

- D_* = Delinquency variables
- S_* = Spend variables
- P_* = Payment variables
- B_* = Balance variables
- R_* = Risk variables

with the following features being categorical:

`['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']`

Your task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric

- `train_data.csv` - training data with multiple statement dates per customer_ID
- `train_labels.csv` - target label for each customer_ID
- `test_data.csv` - corresponding test data; your objective is to predict the target label for each customer_ID
- `sample_submission.csv` - a sample submission file in the correct format

In [ ]:
!pip -q install optbinning

# **<span style="color:#F7B2B0;">Imports</span>**

In [ ]:
import os
import sys
import glob

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
from matplotlib_venn import venn2
import seaborn as sns

from tqdm import tqdm
from itertools import cycle

from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import feature_selection

import lightgbm as lgb
import xgboost as xgb
import catboost as cat

import optbinning

pd.set_option("display.max_columns", None)

plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

# **<span style="color:#F7B2B0;">Data Loading</span>**

reference: https://www.kaggle.com/competitions/amex-default-prediction/discussion/327400

In [ ]:
%%time
train_df = pd.read_feather('../input/amex-default-prediction-feather/train.feather')
test_df = pd.read_feather('../input/amex-default-prediction-feather/test.feather')
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train_df.shape, test_df.shape

In [ ]:
train_df.head()

# **<span style="color:#F7B2B0;">Target Distribution</span>**

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.countplot(x=train_labels.target)
plt.show()

- 0 --> Non Default
- 1 --> Default

# **<span style="color:#F7B2B0;">EDA</span>**

In [ ]:
# checking train test customers

fig, ax = plt.subplots(figsize=(10,5))
set1 = set(train_df.customer_ID.unique())
set2 = set(test_df.customer_ID.unique())

venn2([set1, set2], ('train', 'test'))
plt.show()

- we can observe that non of the user intersect in train test data

In [ ]:
# s_2 date featue
train_df['S_2'] = pd.to_datetime(train_df['S_2'])
test_df['S_2'] = pd.to_datetime(test_df['S_2'])

train_df['S_2'].min(), train_df['S_2'].max()

In [ ]:
test_df['S_2'].min(), test_df['S_2'].max()

- time line also not intersected

In [ ]:
# checking user profiles number of user profiles vs timeline

fig, ax = plt.subplots(figsize=(20,5))
train_df.groupby("S_2")['customer_ID'].count().plot()
plt.title("train profiles vs timeline")
plt.show()

fig, ax = plt.subplots(figsize=(20,5))
test_df.groupby("S_2")['customer_ID'].count().plot()
plt.title("test profiles vs timeline")
plt.show()

- we can observe that train profiles are consistence
- but test profiles increased between oct to apr

In [ ]:
# check each customer profile length

fig, ax = plt.subplots(figsize=(20,5))
sns.countplot(x=train_df.groupby("customer_ID")['customer_ID'].count().values)
plt.title("train customer profile length")
plt.show()

fig, ax = plt.subplots(figsize=(20,5))
sns.countplot(x=test_df.groupby("customer_ID")['customer_ID'].count().values)
plt.title("test customer profile length")
plt.show()

- we can observe that train and test profile lengths are similar distributions

# **<span style="color:#F7B2B0;">Weight of Evidence(WOE)</span>**

The weight of evidence tells the predictive power of an independent variable in relation to the dependent variable. Since it evolved from credit scoring world, it is generally described as a measure of the separation of good and bad customers. "Bad Customers" refers to the customers who defaulted on a loan. and "Good Customers" refers to the customers who paid back loan.

![woe.png](attachment:987d2f10-92e7-4a53-93c1-5a7a0c2836c0.png)

- Distribution of Goods - % of Good Customers in a particular group
- Distribution of Bads - % of Bad Customers in a particular group
- ln - Natural Log

**Steps of Calculating WOE**

1. For a continuous variable, split data into 10 parts (or lesser depending on the distribution).
2. Calculate the number of events and non-events in each group (bin)
3. Calculate the % of events and % of non-events in each group.
4. Calculate WOE by taking natural log of division of % of non-events and % of events

# **<span style="color:#F7B2B0;">Information Value(IV)</span>**

Information value is one of the most useful technique to select important variables in a predictive model. It helps to rank variables on the basis of their importance.

![IV.png](attachment:62ca2622-8754-413d-9f4a-ded369509358.png)

If the IV statistic is:
- Less than 0.02, then the predictor is not useful for modeling (separating the Goods from the Bads)
- 0.02 to 0.1, then the predictor has only a weak relationship to the Goods/Bads odds ratio
- 0.1 to 0.3, then the predictor has a medium strength relationship to the Goods/Bads odds ratio
- 0.3 to 0.5, then the predictor has a strong relationship to the Goods/Bads odds ratio.
- > 0.5, suspicious relationship (Check once)


Reference: https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

# **<span style="color:#F7B2B0;">Feature Selection</span>**

In [ ]:
# Training data preparation
# taking latest profile features for each customer

train_df = train_df.groupby("customer_ID").tail(1).reset_index(drop=True)
test_df = test_df.groupby("customer_ID").tail(1).reset_index(drop=True)

In [ ]:
# Merge with targets
train_df = train_df.merge(train_labels, on='customer_ID', how='left')

In [ ]:
target_col = 'target'
drop_cols = ['customer_ID', 'S_2', target_col]
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train_cols = [col for col in train_df.columns if col not in drop_cols]

## **<span style="color:#F7B2B0;">Information Value (IV)</span>**

- for selecting features we are calculating IV values for each feature
- thankfully `optbinning` will do for us :)

In [ ]:
iv_score_dict = {}
for col in tqdm(train_cols):
    if col in cat_cols:
        optb = optbinning.OptimalBinning(dtype='categorical')
        optb.fit(train_df[col], train_df['target'])
    else:
        optb = optbinning.OptimalBinning(dtype='numerical')
        optb.fit(train_df[col], train_df['target'])
    binning_table = optb.binning_table
    binning_table.build()
    iv_score_dict[col] = binning_table.iv

iv_score_df = pd.Series(iv_score_dict)
iv_score_df.sort_values(ascending=False, inplace=True)

In [ ]:
# top 10 imp iv features
iv_score_df.head(10)

In [ ]:
# iv score vs features
fig, ax = plt.subplots(figsize=(20,5))
iv_score_df.reset_index(drop=True).plot()
plt.show()

- we can observe that top 75 features have > 0.5 IV value
- so those top 75 IV value features are strong predictors

## **<span style="color:#F7B2B0;">Weight of Evidence (WOE)</span>**

- i will try to explan woe values and woe plot for one feature

In [ ]:
col = 'P_2'
optb = optbinning.OptimalBinning(dtype='numerical')
optb.fit(train_df[col], train_df['target'])
binning_table = optb.binning_table
display(binning_table.build())

- `P_2` is a continuous feature, os we splited into 15 bins
- each bin have non-event and event counts and rates
- each bin have WOE and IV values
- for missing values it's created 16th bin

In [ ]:
display(binning_table.plot(metric="woe"))

- from this woe plot we can observe that while increasing bins the event rate decrease
- you can observe that black doted line that is positively correlated with target

In [ ]:
# WOE plots for top 10 features
top10_features = iv_score_df[:10].index.values

for col in top10_features:
    print("-"*100)
    print("="*100)
    print("################ Feature Name : ", col)
    print("\n\n")

    if col in cat_cols:
        optb = optbinning.OptimalBinning(dtype='categorical')
        optb.fit(train_df[col], train_df['target'])
    else:
        optb = optbinning.OptimalBinning(dtype='numerical')
        optb.fit(train_df[col], train_df['target'])

    binning_table = optb.binning_table
    display(binning_table.build())
    display(binning_table.plot(metric="woe"))

- selecting features IV values > 0.5

In [ ]:
selected_features = iv_score_df[iv_score_df > 0.5].index.values
cat_cols = [col for col in cat_cols if col in selected_features]
train_cols = [col for col in train_df.columns if col in selected_features]

## **<span style="color:#F7B2B0;">Correlation Heatmap</span>**

In [ ]:
top_cols = [col for col in selected_features[:20] if col in train_cols]
corr_df = train_df[top_cols].corr()
plt.figure(figsize=(25, 9))
sns.heatmap(corr_df,annot=True ,cmap=sns.color_palette("BrBG",2));
plt.show()

In [ ]:
def drop_feature_selection(row, col, corr, row_iv, col_iv):
    if row_iv >= col_iv:
        return col
    else:
        return row

In [ ]:
cor_matrix = train_df[train_cols].corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool_))
corr_df = upper_tri.stack().reset_index()
corr_df.columns = ['row', 'col', 'corr']
corr_df = corr_df.drop_duplicates()
corr_df = corr_df.sort_values('corr', ascending=False)
corr_df = corr_df.query("corr >= 0.8")
corr_df['row_iv'] = corr_df['row'].map(iv_score_dict)
corr_df['col_iv'] = corr_df['col'].map(iv_score_dict)

corr_df['drop_feature'] = corr_df.apply(lambda x: drop_feature_selection(x['row'], x['col'], x['corr'], x['row_iv'], x['col_iv']), axis=1)

In [ ]:
corr_df

In [ ]:
corr_drop_features = corr_df['drop_feature'].unique().tolist()

# **<span style="color:#F7B2B0;">Modeling</span>**

In [ ]:
# train valid split
train_data, valid_data = model_selection.train_test_split(train_df, test_size=0.3, random_state=42, shuffle=True, stratify=train_df['target'])

train_data.shape, valid_data.shape

In [ ]:
selected_features = [col for col in selected_features if col not in corr_drop_features]
cat_cols = [col for col in cat_cols if col in selected_features]
train_cols = [col for col in train_df.columns if col in selected_features]

In [ ]:
X_train = train_data[train_cols].copy()
y_train = train_data[target_col].copy()

X_valid = valid_data[train_cols].copy()
y_valid = valid_data[target_col].copy()

X_test = test_df[train_cols].copy()

In [ ]:
# binning process

# selection_criteria = {
#     "iv": {"min": 0.02, "max": 1},
#     "quality_score": {"min": 0.01}
# }

binning_process = optbinning.BinningProcess(
    variable_names=train_cols,
    #selection_criteria=selection_criteria,
    categorical_variables=cat_cols
)

In [ ]:
# estimator
estimator = linear_model.LogisticRegression()

In [ ]:
# scorecard
scorecard = optbinning.Scorecard(
    binning_process=binning_process,
    estimator=estimator,
    scaling_method="min_max",
    scaling_method_params={"min": 300, "max": 850},
    # scaling_method = "pdo_odds",
    # scaling_method_params = {"pdo": 20, "odds": 50, "scorecard_points": 100},
    #intercept_based=True,
    #reverse_scorecard=True
)

In [ ]:
# model fitting
scorecard.fit(X_train, y_train)

In [ ]:
# scorecard table
scorecard_df = scorecard.table(style="detailed")

In [ ]:
scorecard_df.head()

In [ ]:
# will try to understand scorecard for one feature

scorecard_df.query("Variable == 'P_2'")

- we can observe that `P_2` feature Points (score).
- while increasing bins the score also increasing
- for example if the user `P_2` value is 0.73 then that user belongs to 7th bin corresponding score is `22.45`

## **<span style="color:#F7B2B0;">Metrics</span>**

In [ ]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [ ]:
train_data['predict_proba'] = scorecard.predict_proba(X_train)[:, 1]
valid_data['predict_proba'] = scorecard.predict_proba(X_valid)[:, 1]

train_score = amex_metric(train_data['target'], train_data['predict_proba'])
valid_score = amex_metric(valid_data['target'], valid_data['predict_proba'])

print("Train Score :", train_score)
print("Valid Score :", valid_score)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, train_data['predict_proba'])
optimal_idx = np.argmax(true_positive_rate - false_positive_rate)
optimal_threshold = thresholds[optimal_idx]
auc_score = metrics.auc(false_positive_rate, true_positive_rate)
print("Train Threshold value is:", optimal_threshold)

false_positive_rate1, true_positive_rate1, thresholds = metrics.roc_curve(y_valid, valid_data['predict_proba'])
optimal_idx = np.argmax(true_positive_rate1 - false_positive_rate1)
optimal_threshold1 = thresholds[optimal_idx]
auc_score1 = metrics.auc(false_positive_rate1, true_positive_rate1)
print("Valid Threshold value is:", optimal_threshold1)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='Binning+LR: Train AUC = {0:.4f}'.format(auc_score))
plt.plot(false_positive_rate1, true_positive_rate1, 'r', label='Binning+LR: Valid AUC = {0:.4f}'.format(auc_score1))
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
train_data['predict'] = (train_data['predict_proba'] > optimal_threshold).astype(int)
valid_data['predict'] = (valid_data['predict_proba'] > optimal_threshold).astype(int)

In [ ]:
conf_mat = metrics.confusion_matrix(train_data['target'], train_data['predict'])
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))

sns.heatmap(conf_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False, ax=ax1)
sns.heatmap(conf_mat/np.sum(conf_mat), annot=True, fmt='.2%', cmap='Blues', ax=ax2)
plt.show()

In [ ]:
conf_mat = metrics.confusion_matrix(valid_data['target'], valid_data['predict'])
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))

sns.heatmap(conf_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False, ax=ax1)
sns.heatmap(conf_mat/np.sum(conf_mat), annot=True, fmt='.2%', cmap='Blues', ax=ax2)
plt.show()

In [ ]:
print(metrics.classification_report(train_data['target'], train_data['predict'], labels=[0, 1]))

In [ ]:
print(metrics.classification_report(valid_data['target'], valid_data['predict'], labels=[0, 1]))

## **<span style="color:#F7B2B0;">Scores</span>**

In [ ]:
train_data['score'] = scorecard.score(X_train)
valid_data['score'] = scorecard.score(X_valid)

In [ ]:
y_test = valid_data['target']
score = valid_data['score']

mask = y_test == 0

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(score[mask], label="non-default", color="b", alpha=0.35)
plt.hist(score[~mask], label="default", color="r", alpha=0.35)
plt.xlabel("score")
plt.legend()
plt.show()

- we can observe that default vs non-default score distribution
- some overlap between 550 to 650
- overall well seperated

In [ ]:
# Plot Distribution of Scores
plt.figure(figsize=(20,10))

plt.hist(score,
         bins=100,
         edgecolor='white',
         color = '#317DC2',
         linewidth=1.2)

#plt.xlim(231,750)
plt.title('Scorecard Distribution', fontweight="bold", fontsize=14)
# plt.axvline(score.mean(), color='k', linestyle='dashed', linewidth=1.5, alpha=0.5)
# plt.text(458, 2970, 'Mean Score: 456', color='red', fontweight='bold', style='italic', fontsize=8)
plt.xlabel('Score')
plt.ylabel('Count');

In [ ]:
# Plot Scores Against Probabilities
plt.figure(figsize=(20,10))

plt.scatter(x=score,
            y=valid_data['predict_proba'],
            #data=scorecard,
            color='#317DC2')

plt.title('Scores by Probability', fontweight="bold", fontsize=14)
plt.xlabel('Score')
plt.ylabel('Probability (Good)');

# **<span style="color:#F7B2B0;">Inference</span>**

In [ ]:
test_df['prediction'] = scorecard.predict_proba(X_test)[:, 1]
sub_df = test_df[['customer_ID', 'prediction']].copy()
sub_df.to_csv("submission.csv", index=False)